In [1]:
from graphframes import GraphFrame
from pyspark.sql.functions import *
from pyspark.sql.types import MapType

In [2]:
inputPathAirports = "/data/students/bigdata-01QYD/Lab10_DBD/airports.csv"
inputPathRoutes = "/data/students/bigdata-01QYD/Lab10_DBD/routes.csv"
#inputPathAirports = "data/airports.csv"
#inputPathRoutes = "data/routes.csv"
outputPath = "resLab10/"

In [3]:
# Read the content of airports.csv
airportsDF = spark.read.load(inputPathAirports,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [4]:
#airportsDF.printSchema()
#airportsDF.show(2)

In [5]:
# Read the content of routes.csv
routesDF = spark.read.load(inputPathRoutes,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [6]:
#routesDF.printSchema()
#routesDF.show(2)

In [7]:
# Remove the records with missing values
cleanedRoutesDF = routesDF\
.filter("airport_source_id IS NOT NULL AND airport_destination_id IS NOT NULL")

In [8]:
# Create the DataFrame associated with the vertexes of the graph
# One vertex per airport
# Cast id from integer to string
vertexesDF = airportsDF.withColumn("id", airportsDF.id.cast("string"))

In [9]:
#vertexesDF.printSchema()
#vertexesDF.show(2)

In [10]:
# Create the DataFrame associated with the edges of the graph
# One vertex per record of cleanedRoutesDF
# We are interersted only in two columns: 
# - airport_source_id
# - airport_destination_id
# Apply the cast operation also on these two columns
# Rename the two colums
# - airport_source_id -> src
# - airport_destination_id -> dst
edgesDF = cleanedRoutesDF.select("airport_source_id","airport_destination_id")\
.withColumn("airport_source_id", cleanedRoutesDF.airport_source_id.cast("string"))\
.withColumn("airport_destination_id", cleanedRoutesDF.airport_destination_id.cast("string"))\
.withColumnRenamed("airport_source_id","src")\
.withColumnRenamed("airport_destination_id","dst")

In [11]:
#edgesDF.printSchema()
#edgesDF.show()

In [12]:
# Create the graph
g = GraphFrame(vertexesDF, edgesDF).cache()

In [13]:
#
#
# Task 1
#
#

# Show on the standard output the top-10 airports by in degree. 
# For each of selected airports show the name of the airport, its ID, and its in degree.

In [14]:
# Retrieve the in degree for each airport
airportIndegree = g.inDegrees

In [15]:
# Create an RDD based on the content of the DataFrame and then apply top
listTop10Airports = airportIndegree.rdd.top(10, key=lambda row: row.inDegree)

# Create a DataFramfrom the python list listTop10Airports
top10AirportsDF = spark.createDataFrame(listTop10Airports, airportIndegree.schema)

In [16]:
#top10AirportsDF.printSchema()
#top10AirportsDF.show()

In [17]:
# Use join to retrieve the name of each airport
top10AirportsNamesDF = top10AirportsDF.join(g.vertices, top10AirportsDF.id == g.vertices.id)\
.select(top10AirportsDF.id, top10AirportsDF.inDegree, g.vertices.name)

In [18]:
top10AirportsNamesDF.show(10, truncate=False)

[Stage 8:==============>                                            (1 + 3) / 4]

+----+--------+------------------------------------------------+
|id  |inDegree|name                                            |
+----+--------+------------------------------------------------+
|3682|911     |Hartsfield Jackson Atlanta International Airport|
|3830|550     |Chicago O'Hare International Airport            |
|3364|534     |Beijing Capital International Airport           |
|507 |522     |London Heathrow Airport                         |
|1382|517     |Charles de Gaulle International Airport         |
|3484|498     |Los Angeles International Airport               |
|340 |493     |Frankfurt am Main Airport                       |
|3670|467     |Dallas Fort Worth International Airport         |
|3797|455     |John F Kennedy International Airport            |
|580 |450     |Amsterdam Airport Schiphol                      |
+----+--------+------------------------------------------------+



In [19]:
#######################################
# Alternative solution based on limit #
#######################################

In [20]:
# Retrieve the in degree for each airport
airportIndegree = g.inDegrees

In [21]:
# Sort airports by in degree and then use limit to select only the first 10 rows
top10AirportsDF = airportIndegree.sort(airportIndegree.inDegree.desc()).limit(10)

In [22]:
#top10AirportsDF.printSchema()
#top10AirportsDF.show()

In [23]:
# Use join to retrieve the name of each airport
top10AirportsNamesDF = top10AirportsDF.join(g.vertices, top10AirportsDF.id == g.vertices.id)\
.select(top10AirportsDF.id, top10AirportsDF.inDegree, g.vertices.name)

In [24]:
top10AirportsNamesDF.show(10, truncate=False)

+----+--------+------------------------------------------------+
|id  |inDegree|name                                            |
+----+--------+------------------------------------------------+
|340 |493     |Frankfurt am Main Airport                       |
|507 |522     |London Heathrow Airport                         |
|580 |450     |Amsterdam Airport Schiphol                      |
|1382|517     |Charles de Gaulle International Airport         |
|3364|534     |Beijing Capital International Airport           |
|3484|498     |Los Angeles International Airport               |
|3670|467     |Dallas Fort Worth International Airport         |
|3682|911     |Hartsfield Jackson Atlanta International Airport|
|3797|455     |John F Kennedy International Airport            |
|3830|550     |Chicago O'Hare International Airport            |
+----+--------+------------------------------------------------+



In [25]:
# End of alternative solution

In [26]:
#
#
# Task 2
#
#

# The Turin airport has id = 1526
# How many airports are reachable from Turin taking exactly 1 flight?
# How many airports are reachable from Turin taking exactly 2 flights?
# How many airports are reachable from Turin taking exactly 3 flights?
# Print the results on the standard output.

In [27]:
# How many airports are reachable from Turin taking exactly 1 flight?
print("How many distinct airports are reachable from Turin taking exactly 1 flight? "\
      +str(g.edges.filter("src=1526").select("dst").distinct().count()))

[Stage 14:=====================================================>(199 + 1) / 200]

How many distinct airports are reachable from Turin taking exactly 1 flight? 29


In [28]:
# How many airports are reachable from Turin taking exactly 2 flights?
# Find the paths airport(Turin) -> airport -> airport
paths2flightsDF = g.find("(a1)-[]->(a2);(a2)-[]->(a3)")

In [29]:
# Select only the flights from Turin and discard those with Turin as final Airport
selectedPathsDF = paths2flightsDF.filter("a1.id=1526 and a3.id<>1526")

In [30]:
print("How many distinct airports are reachable from Turin taking exactly 2 flights? "\
      +str(selectedPathsDF.select("a3").distinct().count()))

24/05/30 19:11:26 WARN util.Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
[Stage 21:===================================================>  (191 + 2) / 200]

How many distinct airports are reachable from Turin taking exactly 2 flights? 589


In [31]:
# How many airports are reachable from Turin taking exactly 3 flights?
# Find the paths airport(Turin) -> airport -> airport
paths3flightsDF = g.find("(a1)-[]->(a2);(a2)-[]->(a3);(a3)-[]->(a4)")

In [32]:
# Select only the flights from Turin and discard those with Turin as final Airport
selectedPathsDF = paths3flightsDF.filter("a1.id=1526 and a4.id<>1526")

In [33]:
print("How many distinct airports are reachable from Turin taking exactly 3 flights? "\
      +str(selectedPathsDF.select("a4").distinct().count()))

[Stage 30:=============================================>        (168 + 2) / 200]

How many distinct airports are reachable from Turin taking exactly 3 flights? 2209


In [34]:
#
#
# Task 3
#
#

# Compute the shortest path length from each airport in the dataset to the Turin airport
# (id = 1526).
# Show the result on the standard output.
# What is the farther airport(s) from Turin, in terms of number of hops?

In [35]:
# Compute the shortest path length from each airport in the dataset to the Turin airport 
# (id = 1526).
shortestPathsDF = g.shortestPaths(["1526"])

In [36]:
#shortestPathsDF.printSchema()
#shortestPathsDF.select("id", "distances").show(10)

In [38]:
# Select only the airport that are reachable from Turin
selectedPathsDF = shortestPathsDF\
.filter("distances['1526'] IS NOT NULL")\
.selectExpr("name", "city", "country", "distances['1526'] as numHops")

In [39]:
# Select the maximum value of numHops
maxNumHops=selectedPathsDF.agg({"numHops":"max"})\
.withColumnRenamed("max(numHops)", "maxHops").first().maxHops

In [40]:
#maxNumHops

In [41]:
# What is the farther airport(s) from Turin, in terms of number of hops?
# For the selected airport(s), show on the standard output 
# its name, its city and country, and the shortest path length to Turin (i.e., number of hops).
selectedPathsDF.filter("numHops="+str(maxNumHops)).show()

+-----------------+---------+-------+-------+
|             name|     city|country|numHops|
+-----------------+---------+-------+-------+
|Peawanuck Airport|Peawanuck| Canada|      8|
+-----------------+---------+-------+-------+



In [42]:
#
#
# Task 4
#
#

# How many connected components of at least two airports are there in the graph?

In [43]:
# Set checkpoint folder
sc.setCheckpointDir("tmp_ckpts")

24/05/30 19:13:56 WARN spark.SparkContext: Spark is not running in local mode, therefore the checkpoint directory must not be on the local filesystem. Directory 'tmp_ckpts' appears to be on the local filesystem.


In [44]:
connectedComponenets = g.connectedComponents()

In [45]:
# Count the number of vertex per component and select only the components with at least two vertexes
connectedComponenetsAtLeatTwo = connectedComponenets.groupBy("component").count().filter("count>1")

In [46]:
# Count the number of selected components
print("How many connected components of at least two airports are there in the graph? "\
      +str(connectedComponenetsAtLeatTwo.count()))


[Stage 579:==========================================>         (165 + 16) / 200]

How many connected components of at least two airports are there in the graph? 7
